In [21]:
import warnings
warnings.filterwarnings('ignore')

import multiprocessing
n_cores = multiprocessing.cpu_count()

import sys
import os
import copy 

sys.path.insert(1, os.path.realpath(os.path.pardir))

import pandas as pd
import numpy as np

In [22]:

grn_filenames = ("../data/networks/inferelator/signed_network.tsv",
                 "../data/networks/inferelator/CSTARVE_signed_network.tsv") 


tissue = 'yeast'
target = 'wildVScstarve'
network_inf_method = 'INFERELATOR'
gsea_parent_folder_name = "INFERE_CSTARVE_UASE_n2v2r"



#do the data have index and header??
## if it is .csv, most likely the sep = ","
#  if it is .tsv, most likely the sep = "\t"
index_col = 0
header = 0
sep = '\t'


#load the gene regulatory networks into PANDAS dataframes and then a list
# the yeast inferelator data have TFs in columns, so we transpose to have them on rows ()
grns = []
for grn_filename in grn_filenames:
    grn_pd = pd.read_csv(grn_filename,  index_col=index_col, header=header, sep=sep).T
    grns.append(grn_pd)

In [23]:
## use commmon genes in case and control

control = grns[0].copy()
case = grns[1].copy()

control_columns = control.columns.to_list()
control_rows = control.index.to_list()

case_columns = case.columns.to_list()
case_rows = case.index.to_list()

case_net_columns = grns[1].columns.to_list()
case_net_rows = grns[1].index.to_list()

genes_in_control_not_in_case = set(control_columns) - set(case_net_columns)
genes_in_case_not_in_control = set(case_columns) - set(control_columns)

common_cols = list(set(case_net_columns).intersection(control_columns))
common_rows = list(set(case_net_rows).intersection(control_rows))

case = case.loc[:,common_cols]
control = control.loc[:,common_cols]

case = case.loc[common_rows,:]
control = control.loc[common_rows,:]

grns[0] = control
grns[1] = case

In [24]:
## network transformation and preparation for n2v2r
## be mindful of the transformations and their consequences e.g., n2v2r does not accept negative weights

from node2vec2rank.utils import network_transform

row_genes = grns[0].index.to_numpy()
col_genes = grns[0].columns.to_numpy()

yeast_map_fn = '../data/gene_set_libraries/yeast/yeast_orf_to_symbol_mapping.tsv'
yeast_map = pd.read_csv(yeast_map_fn, sep="\t")
orf2symbol = {i['orf']:i['name'] for k,i in yeast_map.iterrows()}

## use names for TFs to ensure all nodes have unique names (i.e., a TF and a gene having the same ORF will not be a problem)
row_genes_name = [orf2symbol[x] for x in row_genes]

total_genes = np.append(row_genes_name, col_genes)
total_genes = np.unique(total_genes)

num_rows = np.size(row_genes_name)
num_cols = np.size(col_genes)
num_total = np.size(total_genes)


print(f"There are {num_rows} row genes, {num_cols} column genes, and {num_total} unique in first (anchor) graph")

## network transformation

## parameteres for network transformation
# make the network binary
binarize_network = False

absolute = True

# only keeps the top %
top_percent_keep = 100

#transform the networks 
grns_transformed = []
for grn in grns:
    grns_transformed.append(network_transform(grn.to_numpy(), 
    binarize=binarize_network,
    absolute=absolute,
    threshold=0, 
    top_percent_keep=top_percent_keep,
    project_unipartite=True))


There are 129 row genes, 1254 column genes, and 1383 unique in first (anchor) graph


In [25]:
## get DeDi 
## map gene ids to names if necessary

control_net_adj = grns[0]
case_net_adj =  grns[1] 

net_one_adj_indegree = control_net_adj.sum(axis=0).to_numpy()
net_two_adj_indegree = case_net_adj.sum(axis=0).to_numpy()

DeDi = net_one_adj_indegree - net_two_adj_indegree
absDeDi = np.abs(DeDi)

DeDi_genes_orf = control_net_adj.columns.to_list()

# map orf to gene name
yeast_map_fn = '../data/gene_set_libraries/yeast/yeast_orf_to_symbol_mapping.tsv'
yeast_map = pd.read_csv(yeast_map_fn, sep="\t")
orf2symbol = {i['orf']:i['name'] for k,i in yeast_map.iterrows()}
DeDi_genes_name = [orf2symbol[x] for x in DeDi_genes_orf]

DeDi_data_dict = {"genes" :DeDi_genes_name, "DeDi": DeDi, "absDeDi": absDeDi}

DeDi_data_pd = pd.DataFrame(DeDi_data_dict, index=DeDi_genes_name)
DeDi_data_pd.sort_values(by='absDeDi', ascending=False, inplace=True)

In [26]:
from node2vec2rank.model import n2v2r

import json

from scipy.sparse import csc_matrix


#read the config file
config = json.load(open('../configs/config.json', 'r'))

config = {param: value for section, params in config.items()
          for param, value in params.items()}

##the dictionary for mapping indices to gene names
node_names = DeDi_genes_name

graphs=[]

graphs.append(grns_transformed[0])
graphs.append(grns_transformed[1])

model = n2v2r(graphs=graphs, config=config, node_names=node_names)
rankings = model.fit_transform_rank()

borda_rankings = model.aggregate_transform()

signed_rankings = model.signed_transform(DeDi_data_pd.iloc[:,1])


Running n2v2r with dimensions [2, 4, 6, 8, 10, 12, 14, 16] and distance metrics ['euclidean', 'cosine', 'cityblock', 'correlation', 'chebyshev'] ...
        Finished UASE embedding in 1.09 seconds
 Ranking ...
        Finished ranking in 0.59 seconds
    n2v2r took 1.6800000000000002 seconds

Rank aggregation with Borda ...
    Finished aggregation in 0.88 seconds

Signed transformation ...
    Finished signed transformation in 0.46 seconds


In [27]:
n2v2r_ranking_pd = rankings[0]
n2v2r_borda_ranking_pd = borda_rankings[0]
n2v2r_DeDi_ranking_pd = signed_rankings[0]
n2v2r_borda_DeDi_ranking_pd = model.aggregate_signed_ranks_sequence[0]

In [28]:
from node2vec2rank.visualization_utils import dim_reduction, plot_embeddings

algorithm = 'pca'
n_components = 2

first_embeddings = model.node_embeddings[0]
second_embeddings = model.node_embeddings[1]
concat_embeddings = np.append(first_embeddings, second_embeddings, axis=0)

first_embeddings_red = dim_reduction(first_embeddings[:,:6], algorithm=algorithm, n_components=n_components)
second_embeddings_red = dim_reduction(second_embeddings[:,:6], algorithm=algorithm, n_components=n_components)
concat_embeddings_red = dim_reduction(concat_embeddings[:,:6], algorithm=algorithm, n_components=n_components)

plot_embeddings(first_embeddings_red, color_type='numeric', color = n2v2r_borda_ranking_pd.loc[node_names,'borda_ranks'], names=node_names)
plot_embeddings(second_embeddings_red, color_type='numeric', color = n2v2r_borda_ranking_pd.loc[node_names,'borda_ranks'], names=node_names)

num_nodes = first_embeddings_red.shape[0]
color_one = np.zeros(num_nodes)
color_two = np.ones(num_nodes)
color_concat = np.append(color_one, color_two, axis=0)

plot_embeddings(concat_embeddings_red, color=color_concat, names = np.append(node_names,node_names))

In [29]:
# run enrich GSEA
from node2vec2rank.post_utils import enrich_gsea, read_gmt
from itertools import chain
import os

save_results = True
save_results_notes = ''

# read the geneset libraries
kegg_pathway_fn = '../data/gene_set_libraries/yeast/KEGG_2018_yeast.gmt'
gobp_pathway_fn = '../data/gene_set_libraries/yeast/GO_Biological_Process_2018_yeast.gmt'

# network_background or pathway_background for enrichment
# network will use the genes in the network only, while pathway will use all the genes in the pathways
# network is "more fair" but will find less things in small networks
background = 'pathway_background'
organism = 'yeast'

enrich_padj_cutoff = 0.1
# take the top k percentage of the ranking for enrichment
top_k_percent = 5

if background == 'network_background':
    kegg_background = n2v2r_ranking_pd.index.to_list()
    gobp_background = n2v2r_ranking_pd.index.to_list()
elif background == 'pathway_background':
    kegg_dict = read_gmt(kegg_pathway_fn)
    kegg_background = list(set(chain.from_iterable(kegg_dict.values())))
    gobp_dict = read_gmt(gobp_pathway_fn)
    gobp_background = list(set(chain.from_iterable(gobp_dict.values())))
else:
    raise Exception("Enrichment background not properly set")

n2v2r_enr_KEGG_pd = enrich_gsea(n2v2r_ranking_pd, kegg_pathway_fn, background=kegg_background,
                                          enrich_padj_cutoff=enrich_padj_cutoff, enrich_quantile_cutoff=1-top_k_percent/100, organism=organism)

n2v2r_enr_GOBP_pd = enrich_gsea(n2v2r_ranking_pd, gobp_pathway_fn, background=gobp_background,
                                          enrich_padj_cutoff=enrich_padj_cutoff, enrich_quantile_cutoff=top_k_percent/100, organism=organism)

borda_enr_KEGG_pd = enrich_gsea(n2v2r_borda_ranking_pd, kegg_pathway_fn, background=kegg_background,
                                enrich_padj_cutoff=enrich_padj_cutoff, enrich_quantile_cutoff=1-top_k_percent/100, organism=organism)

borda_enr_GOBP_pd = enrich_gsea(n2v2r_borda_ranking_pd, gobp_pathway_fn, background=gobp_background,
                                enrich_padj_cutoff=enrich_padj_cutoff, enrich_quantile_cutoff=1-top_k_percent/100, organism=organism)

absDeDi_enr_KEGG_pd = enrich_gsea(DeDi_data_pd[['absDeDi']], kegg_pathway_fn, background=kegg_background,
                                  enrich_padj_cutoff=enrich_padj_cutoff, enrich_quantile_cutoff=1-top_k_percent/100, organism=organism)

absDeDi_enr_GOBP_pd = enrich_gsea(DeDi_data_pd[['absDeDi']], gobp_pathway_fn, background=gobp_background,
                                  enrich_padj_cutoff=enrich_padj_cutoff, enrich_quantile_cutoff=1-top_k_percent/100, organism=organism)

if save_results:
    path = '../results/results_gsea/' + gsea_parent_folder_name
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)

    n2v2r_enr_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                                       "_n2v2r"+"_consensus_enr_KEGG_"+background+"_top"+str(top_k_percent)+"_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_enr_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                                       "_n2v2r"+"_consensus_enr_GOBP_"+background+"_top"+str(top_k_percent)+"_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    borda_enr_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target+"_n2v2r" +
                             "_borda_enr_KEGG_"+background+"_top"+str(top_k_percent)+"_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    borda_enr_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target+"_n2v2r" +
                             "_borda_enr_GOBP_"+background+"_top"+str(top_k_percent)+"_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    absDeDi_enr_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                               "_absDeDi"+"_enr_KEGG_"+background+"_top"+str(top_k_percent)+"_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    absDeDi_enr_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                               "_absDeDi"+"_enr_GOBP_"+background+"_top"+str(top_k_percent)+"_"+save_results_notes+".tsv", header=True, index=None, sep='\t')


combo: dim-2_distance-euclidean with 4 found
combo: dim-2_distance-cityblock with 1 found
combo: dim-2_distance-chebyshev with 4 found
combo: dim-4_distance-euclidean with 5 found
combo: dim-4_distance-cosine with 1 found
combo: dim-4_distance-cityblock with 5 found
combo: dim-4_distance-correlation with 1 found
combo: dim-4_distance-chebyshev with 4 found
combo: dim-6_distance-euclidean with 5 found
combo: dim-6_distance-cityblock with 4 found
combo: dim-6_distance-chebyshev with 4 found
combo: dim-8_distance-euclidean with 2 found
combo: dim-8_distance-chebyshev with 5 found
combo: dim-10_distance-euclidean with 4 found
combo: dim-10_distance-cityblock with 3 found
combo: dim-10_distance-chebyshev with 5 found
combo: dim-12_distance-euclidean with 4 found
combo: dim-12_distance-cityblock with 1 found
combo: dim-12_distance-chebyshev with 5 found
combo: dim-14_distance-euclidean with 4 found
combo: dim-14_distance-cityblock with 1 found
combo: dim-14_distance-chebyshev with 8 found
co

In [30]:
## run prerank GSEA
from node2vec2rank.post_utils import prerank_gsea

save_results = True
save_results_notes = ''

# read the geneset libraries
kegg_pathway_fn = '../data/gene_set_libraries/yeast/KEGG_2018_yeast.gmt'
gobp_pathway_fn = '../data/gene_set_libraries/yeast/GO_Biological_Process_2018_yeast.gmt'

prerank_padj_cutoff = 0.25
prerank_weight = 0
prerank_min_path_size = 5
prerank_max_path_size = 1500
prerank_num_perms = 1000

n2v2r_pre_KEGG_pd = prerank_gsea(n2v2r_ranking_pd, kegg_pathway_fn, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                           prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

n2v2r_pre_GOBP_pd = prerank_gsea(n2v2r_ranking_pd, gobp_pathway_fn, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                           prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

borda_pre_KEGG_pd = prerank_gsea(n2v2r_borda_ranking_pd, kegg_pathway_fn, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                 prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

borda_pre_GOBP_pd = prerank_gsea(n2v2r_borda_ranking_pd, gobp_pathway_fn, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                 prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

absDeDi_pre_KEGG_pd = prerank_gsea(DeDi_data_pd[['absDeDi']], kegg_pathway_fn, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                   prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

absDeDi_pre_GOBP_pd = prerank_gsea(DeDi_data_pd[['absDeDi']], gobp_pathway_fn, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                   prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

DeDi_pre_KEGG_pd = prerank_gsea(DeDi_data_pd[['DeDi']], kegg_pathway_fn, one_sided=False, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

DeDi_pre_GOBP_pd = prerank_gsea(DeDi_data_pd[['DeDi']], gobp_pathway_fn, one_sided=False, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

n2v2r_borda_DeDi_pre_KEGG_pd = prerank_gsea(n2v2r_borda_DeDi_ranking_pd, kegg_pathway_fn, one_sided=False, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

n2v2r_borda_DeDi_pre_GOBP_pd = prerank_gsea(n2v2r_borda_DeDi_ranking_pd, gobp_pathway_fn, one_sided=False, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

n2v2r_DeDi_pre_KEGG_pd = prerank_gsea(n2v2r_DeDi_ranking_pd, kegg_pathway_fn, one_sided=False, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

n2v2r_DeDi_pre_GOBP_pd = prerank_gsea(n2v2r_DeDi_ranking_pd, gobp_pathway_fn, one_sided=False, prerank_padj_cutoff=prerank_padj_cutoff, prerank_weight=prerank_weight,
                                prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

if save_results:
    path = '../results/results_gsea/' + gsea_parent_folder_name
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)

    n2v2r_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                                       "_n2v2r"+"_consensus_prerank_KEGG_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                                       "_n2v2r"+"_consensus_prerank_GOBP_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    borda_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target+"_n2v2r" +
                             "_borda_prerank_KEGG_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    borda_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target+"_n2v2r" +
                             "_borda_prerank_GOBP_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    absDeDi_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                               "_absDeDi"+"_prerank_KEGG_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    absDeDi_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                               "_absDeDi"+"_prerank_GOBP_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    DeDi_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                            "_DeDi"+"_prerank_KEGG_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    DeDi_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                            "_DeDi"+"_prerank_GOBP_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_borda_DeDi_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                            "_n2v2r_borda_DeDi"+"_prerank_KEGG_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_borda_DeDi_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                            "_n2v2r_borda_DeDi"+"_prerank_GOBP_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_DeDi_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                            "_n2v2r_chimera"+"_prerank_KEGG_"+save_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_DeDi_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                            "_n2v2r_chimera"+"_prerank_GOBP_"+save_results_notes+".tsv", header=True, index=None, sep='\t')


2023-04-23 01:10:25,832 [WARNING] Duplicated values found in preranked stats: 0.16% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2023-04-23 01:10:25,973 [WARNING] Duplicated values found in preranked stats: 0.24% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2023-04-23 01:10:26,112 [WARNING] Duplicated values found in preranked stats: 0.24% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


combo: dim-2_distance-euclidean with 1 found


2023-04-23 01:10:26,253 [WARNING] Duplicated values found in preranked stats: 0.24% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2023-04-23 01:10:26,391 [WARNING] Duplicated values found in preranked stats: 0.24% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


combo: dim-2_distance-cityblock with 2 found


2023-04-23 01:10:26,669 [WARNING] Duplicated values found in preranked stats: 0.32% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2023-04-23 01:10:26,806 [WARNING] Duplicated values found in preranked stats: 0.08% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


...

combo: dim-16_distance-cosine with 29 found


2023-04-23 01:11:28,023 [WARNING] Duplicated values found in preranked stats: 0.32% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


combo: dim-16_distance-cityblock with 32 found


2023-04-23 01:11:28,501 [WARNING] Duplicated values found in preranked stats: 0.08% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


combo: dim-16_distance-correlation with 29 found
combo: dim-16_distance-chebyshev with 28 found
